In [1]:
import numpy as np
import matplotlib.pylab as plt
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
text = '''
Cats, cats, everywhere
Furry balls without a care
Purring, meowing, licking fur
Hunting mice, they always purr

Cats, cats, on the prowl
Jumping high, never a scowl
Whiskers twitching, eyes alert
Tail in air, ready to assert

Cats, cats, so much fun
Cuddling close in the sun
Stretching out, napping long
Playing with string, never wrong

Cats, cats, always cool
Lapping milk, acting like a fool
Mysterious, charming, full of grace
Cats are simply ace

Cats, cats, with silky fur
Making biscuits, they always purr
Sitting high, looking down
Claiming everything as their crown

Cats, cats, with eyes so bright
Chasing shadows, day or night
Curled up warm, on your lap
Purring gently, taking a nap

Cats, cats, with playful paws
Hiding, stalking, never pause
Jumping, leaping, so agile
Graceful creatures, never fragile

Cats, cats, our feline friends
Bringing joy that never ends
Loving us, without a doubt
Cats are what life's all about

Cats, cats, everywhere I see
Furry creatures, cute as can be
Rubbing against our legs
Asking for treats, without begs

Cats, cats, with their regal stance
Graceful movements, they enhance
But we love them all the same
Our little friends, never tame

Cats, cats, so full of love
Watching over us from above
Protecting us from any harm
Always there, with their charm

Cats, cats, with their curious ways
Exploring nooks, and hiding in bays
Living life with style and grace
Cats are always in first place

Cats, cats, so full of fun
Chasing toys, never done
Hiding in boxes, or paper bags
Making us laugh, never drags

Cats, cats, with their own minds
Sitting in the sun, never blinds
Chasing strings, and balls of yarn
They never tire, oh what a charm

Cats, cats, with calming purrs
Cuddling close, to be yours
Giving love, without any fuss
Their presence, a comfort to us

Cats, cats, always at ease
Living life, as they please
Bringing joy, to all they meet
Cats, our furry friends, so sweet

Cats, cats, with eyes so bright
Guiding us through the darkest night
Purring softly, by our side
Comforting us, as we hide

Cats, cats, with softest fur
Nuzzling close, making a purr
In our lap, they take a rest
We're lucky to have, such a guest

Cats, cats, with their playful ways
Entertaining us, on the laziest days
Chasing shadows, or a feather
Making us smile, always together

Cats, cats, with hearts so pure
Bringing love, that will endure
Their presence, a blessing indeed
Cats, our friends, we shall never need

Cats, cats, with their little quirks
Scratching posts, and tiny perks
Licking paws, cleaning their face
Chasing tails, all over the place

Cats, cats, with their playful hearts
Chasing toys, and little carts
Their antics, bringing us joy
Cats, our little angels, oh so coy

Cats, cats, with their gentle souls
Lifting spirits, making us whole
In their eyes, we see the light
Bringing peace, that feels so right

Cats, cats, with their gentle purr
Calming us, when we're feeling a stir
Snuggling close, to keep us warm
Cats, our little cuddle storm

Cats, cats, with their playful heart
Jumping high, right from the start
Bouncing around, like little springs
Cats, our little entertainers, with wings

Cats, cats, with their loving grace
Their soft purrs, caress our face
In their embrace, we feel at peace
Cats, our little comfort, never to cease

Cats, cats, with their loving ways
Cuddling close, on the darkest days
In the garden, or up in a tree
Cats, our little explorers, always free

'''

text = text.lower()
chars = sorted(list(set(text)))
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
data = [stoi[c] for c in text]
vocab_size = len(chars)

ins = 16
outs = vocab_size
nodes = 200
lr = 0.002

n_emb = 32
embed = torch.randn(vocab_size, n_emb)
print(f"embed size {embed.shape}")
pos = torch.randn(ins, n_emb)
print(f"pos pos {pos.shape}")

data = torch.tensor(data).long()

embed size torch.Size([30, 32])
pos pos torch.Size([16, 32])


In [3]:
params = [] 
def weights(ins, outs):
    ws = torch.randn(ins, outs)*0.1
    ws.requires_grad_(True)
    params.append(ws)
    return ws

class Head():
    def __init__(self):
        self.wv = weights(n_emb, n_emb//4)
        self.wk = weights(n_emb, n_emb//4)
        self.wq = weights(n_emb, n_emb//4)
    
    def forward(self, x):
        v = x @ self.wv
        k = x @ self.wk
        q = x @ self.wq
        attn = (q @ k.transpose(-2, -1)) / k.shape[0]**0.5
        tril = torch.tril(attn)
        tril = tril.masked_fill(tril == 0, -1e10)
        rew = F.softmax(tril, dim=-1)
        x = rew @ v
        return x        


In [4]:
class Model():
    def __init__(self):
        self.heads = [Head(), Head(), Head(), Head()]
        self.w0 = weights(n_emb, nodes)
        self.w1 = weights(nodes, nodes)
        self.w2 = weights(nodes, outs)

    def forward(self, x):
        #print(f"x shape 0{x.shape}")
        #print(f"embed[x] {embed[x].shape}  pos shape  {pos.shape}")
        # x broadcast to 3d *100
        x = embed[x] + pos
        #print(f"x shape 1{x.shape}")  x shape 0torch.Size([100, 16])  --> [100, 16, 32]
        # batch inputs become embedings , so x to 3D
        x = torch.cat([head.forward(x) for head in self.heads], dim=-1)
        x = torch.relu(x @ self.w0)
        x = torch.relu(x @ self.w1)
        yh = x @ self.w2 
        return yh   ## the predict



In [5]:
model = Model()
optimizer = torch.optim.Adam(params, lr)

def generate_text(s, length):
    print("generated text:")
    print("---------------------")
    gen_text = ""
    for i in range(length):
        yh = model.forward(s)
        prob = F.softmax(yh[-1, :], dim=0)
        # pred = torch.argmax(yh).item()
        #print(f"*** prob {prob}, sample nomial {torch.multinomial(prob, num_samples=1)}")
        pred = torch.multinomial(prob, num_samples=1).item() 
        ## update the input tokens
        s = torch.roll(s, -1)
        s[-1] = pred
        gen_text += itos[pred]
    
    print(gen_text)


b = torch.randint(len(data)-ins, (100,))
print(b) 
print("b shape:", b.shape)
# batch

for it in range(5000):
    
    b = torch.randint(len(data)-ins, (100,))
    xs = torch.stack([data[i:i+ins] for i in b])
    ys = torch.stack([data[i+1:i+ins+1] for i in b])

    #print(f"xs shape{xs.shape}")  [100, 16]
    yh = model.forward(xs)
    # view change tensor shape to x*size
    loss = F.cross_entropy(yh.view(-1, vocab_size) , ys.long().view(-1)) 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    e = loss.item()
    if it%1000==0:
        print(it, "loss:", e)
            
generate_text(xs[0], 1000)

tensor([2790, 1561, 2457, 3004, 2434, 1314,  369, 2512, 3216, 1031, 2087, 2054,
         532, 1558, 1364, 2307, 1048,   36,  602, 1607,  317, 1027, 1841,   74,
        1190, 3041,  409, 1348,   12, 2740,  879, 3246, 1151, 3177, 2839,  798,
        2145,  602, 3163, 1162,  993, 2613, 3364, 2808,    2, 1542, 3371, 2304,
         197, 1778, 2234, 3300, 1144, 2668,   19,  899, 2737, 3083, 1898,  285,
        2478, 3190,  634, 2826,  709, 2480,  874, 1074, 1570, 1978, 1336,  976,
         588, 1269,  750,  291, 1333,  762, 1422, 1744,   98, 2555, 2053, 1947,
        1687, 1015, 3373, 3028, 1916, 2203, 1336,  458, 1116, 2732,  823,  553,
        1931, 2346, 1538, 2909])
b shape: torch.Size([100])
embed[x] torch.Size([100, 16, 32])  pos shape  torch.Size([16, 32])
0 loss: 3.4211678504943848
embed[x] torch.Size([100, 16, 32])  pos shape  torch.Size([16, 32])
embed[x] torch.Size([100, 16, 32])  pos shape  torch.Size([16, 32])
embed[x] torch.Size([100, 16, 32])  pos shape  torch.Size([16, 32])
e

KeyboardInterrupt: 